In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm

from scipy.sparse import lil_matrix

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.impute import SimpleImputer

In [2]:
from developed_methods import *

In [3]:
import scipy as sp
from scipy.stats import chi2

def mahalanobis_method(df):
    #M-Distance
    x_minus_mu = df - np.mean(df)
    cov = np.cov(df.values.T)                           #Covariance
    inv_covmat = sp.linalg.inv(cov)                     #Inverse covariance
    left_term = np.dot(x_minus_mu, inv_covmat) 
    mahal = np.dot(left_term, x_minus_mu.T)
    md = np.sqrt(mahal.diagonal())
    
    #Flag as outlier
    outlier = []
    #Cut-off point
    C = np.sqrt(chi2.ppf((1-0.001), df=df.shape[1]))    #degrees of freedom = number of variables
    for index, value in enumerate(md):
        if value > C:
            outlier.append(index)
        else:
            continue
    return outlier, md

# save the predicted ratings to csv file
def save_csv(df, folder_path, method):
    nowTime = datetime.now().strftime("%Y-%m-%d_%H-%M")
    fileName = "{folder_path}/{method}_{nowTime}.csv".format(folder_path = folder_path, method = method, nowTime = nowTime)
    df.to_csv(fileName, index = False)


In [4]:
# Load dataset
train_rating = pd.read_csv("../data/train_rating.csv")
test_pair = pd.read_csv("../data/test_pair.csv")

item_feat = pd.read_csv("../data/item_feats.csv")
user_feat = pd.read_csv("../data/user_feats.csv")

sub = pd.read_csv('../predict/sample_submission.csv')

In [5]:
# UserID
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(np.append(train_rating['UserId'], test_pair["UserId"]), user_feat["UserId"]))

user_feat['UserId'] = le_user.transform(user_feat["UserId"])
test_pair["UserId"] = le_user.transform(test_pair["UserId"])
train_rating['UserId'] = le_user.transform(train_rating["UserId"])

# ItemID
le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(np.append(train_rating['ItemId'], test_pair["ItemId"]), item_feat["ItemId"]))

item_feat['ItemId'] = le_item.transform(item_feat["ItemId"])
test_pair["ItemId"] = le_item.transform(test_pair["ItemId"])
train_rating['ItemId'] = le_item.transform(train_rating["ItemId"])

#Inf value
user_feat.loc[np.isinf(user_feat['V1']),'V1']=-3
item_feat.loc[np.isinf(item_feat['V2']),'V2']=2

# Missing data
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(user_feat['V1'].values.reshape(-1, 1))
user_feat['V1'] = imp_mean.transform(user_feat['V1'].values.reshape(-1, 1))

In [6]:
# tran_pair, train_rating
train_pairs = train_rating[['UserId', 'ItemId']].values
train_ratings = train_rating['rating'].values
train_pair=train_rating.drop(columns='rating')

# test_pair
test_pairs = test_pair[['UserId', 'ItemId']].values

# number of users and items
# n_user, n_item = len(le_item.classes_), len(le_item.classes_)

In [7]:
n_user, n_item = max(train_pairs[:,0].max(), test_pairs[:,0].max())+1, max(train_pairs[:,1].max(), test_pairs[:,1].max())+1

In [8]:
class min_max_adj:
    def __init__(self, train_rating):
        self.min = np.min(train_rating)
        self.max = np.max(train_rating)
        self.true_rating = train_rating
    
    def adjust(self, pred_rating):
        pred_rating_adjusted = pred_rating.copy()
        pred_rating_adjusted[pred_rating > self.max] = self.max
        pred_rating_adjusted[pred_rating < self.min] = self.min
        return pred_rating_adjusted

    def rmse(self, pred_rating):
        return np.sqrt(np.mean((pred_rating - self.true_rating)**2))

adjustment = min_max_adj(train_rating["rating"])
print("Minmium and maximum:", [adjustment.min, adjustment.max])

Minmium and maximum: [0.0, 5.0]


In [9]:
## generate cont feats for users
user_pd = pd.merge(left=train_rating.groupby('UserId')['rating'].mean(), 
				   right=train_rating.groupby('UserId')['rating'].count(), on='UserId')
user_pd.columns = ['rating_mean', 'rating_count']
user_pd = pd.merge(left = user_feat, right = user_pd, on = "UserId")

## generate cont feats for items
item_rating_pd = pd.merge(left=train_rating.groupby('ItemId')['rating'].mean(), 
						  right=train_rating.groupby('ItemId')['rating'].count(), on='ItemId')
item_rating_pd.columns	= ['rating_mean', 'rating_count']
item_pd = pd.merge(left=item_feat, right=item_rating_pd, on='ItemId')

print('#######################################################')
print('########## 10 random samples for users feats ##########')
print('#######################################################')

print(user_pd.sample(10))
print('#######################################################')
print('########## 10 random samples for items feats ##########')
print('#######################################################')

print(item_pd.sample(10))

#######################################################
########## 10 random samples for users feats ##########
#######################################################
      UserId        V1    V2    V3   V4  rating_mean  rating_count
4925   16909  3.492023  2195  1284  325     2.500000             3
5745    1047  3.423729  7314  1079   56     0.000000             1
5406   17120  3.011002  1555   938  115     0.000000             1
1637    6011  3.422410  1119  1555  330     1.166667           144
3595    1426  3.697473  3306  1422  330     3.750000             2
1442   15666  2.932806  6220   262  330     3.000000             1
5503      83  3.995718  2920  1422  330     4.000000             3
3473    4031  3.719076  7504  1493  290     0.763889            36
2183    6370  3.314404  1084   938  265     3.625000             8
4759    4330  4.053854  4640   890  330     5.000000             1
#######################################################
########## 10 random samples for items 

In [10]:
## pre-processing for users
user_cont = ["V1", "V2", "V3", "V4", "rating_mean", "rating_count"]
user_pd[user_cont] = preprocessing.StandardScaler().fit_transform(user_pd[user_cont])

## pre-processing for item
item_cont = ["V1", "V2", "V3", "rating_mean", "rating_count"]
item_pd[item_cont] = preprocessing.StandardScaler().fit_transform(item_pd[item_cont])


user_pd = user_pd.set_index('UserId', drop=True)
item_pd = item_pd.set_index('ItemId', drop=True)

print('#######################################################')
print('########## 10 random samples for users feats ##########')
print('#######################################################')
print(user_pd.sample(10))

print('#######################################################')
print('########## 10 random samples for items feats ##########')
print('#######################################################')
print(item_pd.sample(10))

#######################################################
########## 10 random samples for users feats ##########
#######################################################
              V1        V2        V3        V4  rating_mean  rating_count
UserId                                                                   
8195   -0.050470 -0.757371  0.649809  0.611353     1.035314     -0.125928
7211   -0.050470  1.057717  0.119038  0.611353     0.673439     -0.062890
11459   0.152783 -1.571060 -0.608316  0.611353     1.614313     -0.125928
15114  -0.483688 -0.212080  1.672037 -2.265552    -1.280683     -0.104915
17120  -1.046634 -1.094143  0.324357 -1.262994    -1.280683     -0.125928
601     1.016787 -0.771810  0.752469  0.611353     1.035314     -0.125928
13722   0.209898  0.300086  0.870418 -2.073758    -1.280683     -0.125928
4744    0.275941 -0.837635 -0.368049  0.611353    -1.280683     -0.125928
16238   0.817224  0.017248 -0.765582  0.567764     1.035314     -0.125928
21262  -0.265473 -

In [11]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input, Dropout, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from IPython.display import SVG
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [12]:
class SideNCF(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(SideNCF, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.itme_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )

        self.concatenate = layers.Concatenate()
        self.dense1 = layers.Dense(100, name='fc-1', activation='relu')
        self.dense2 = layers.Dense(50, name='fc-2', activation='relu')
        self.dense3 = layers.Dense(1, name='fc-3', activation='relu')

    def call(self, inputs):
        cont_feats = inputs[0]
        cate_feats = inputs[1]

        user_vector = self.user_embedding(cate_feats[:,0])
        itme_vector = self.itme_embedding(cate_feats[:,1])

        concatted_vec = self.concatenate([cont_feats, user_vector, itme_vector])
        fc_1 = self.dense1(concatted_vec)
        fc_2 = self.dense2(fc_1)
        fc_3 = self.dense3(fc_2)
        return fc_3

In [13]:
model = SideNCF(num_users=n_user, num_items=n_item, embedding_size=50)

metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

In [14]:
## find the continuous features and categorical features for user and item, respectively
item_cont, item_cate = ["V1", "V2", "V3", 'rating_mean', 'rating_count'], ['ItemId']
user_cont, user_cate = ["V1", "V2", "V3", "V4", 'rating_mean', 'rating_count'], ['UserId']

train_cont_feats = np.hstack((user_pd.loc[train_pairs[:,0]][user_cont], item_pd.loc[train_pairs[:,1]][item_cont]))
train_cate_feats = train_pairs.copy()

In [15]:
test_cont_feats = np.hstack((user_pd.loc[test_pairs[:,0]][user_cont], item_pd.loc[test_pairs[:,1]][item_cont]))
test_cate_feats = test_pairs.copy()

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([ 1018, 25486,   347, 25355,  8906,\n            ...\n             4722, 16336, 26116, 23128, 16653],\n           dtype='int64', name='UserId', length=1618). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [16]:
callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=[train_cont_feats, train_cate_feats],
    y=train_ratings,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=.2,
)

Epoch 1/50
504/504 [==============================] - 15s 27ms/step - loss: 1.9267 - mae: 0.9200 - rmse: 1.3649 - val_loss: 1.6984 - val_mae: 0.8651 - val_rmse: 1.2883
Epoch 2/50
504/504 [==============================] - 14s 27ms/step - loss: 1.4699 - mae: 0.7507 - rmse: 1.1948 - val_loss: 1.4616 - val_mae: 0.7551 - val_rmse: 1.1921
Epoch 3/50
504/504 [==============================] - 14s 27ms/step - loss: 1.4296 - mae: 0.7254 - rmse: 1.1755 - val_loss: 1.4809 - val_mae: 0.7371 - val_rmse: 1.1962
Epoch 4/50
504/504 [==============================] - 14s 29ms/step - loss: 1.4268 - mae: 0.7156 - rmse: 1.1705 - val_loss: 1.5288 - val_mae: 0.7570 - val_rmse: 1.2111
Epoch 5/50
504/504 [==============================] - 15s 30ms/step - loss: 1.4226 - mae: 0.7088 - rmse: 1.1654 - val_loss: 1.5233 - val_mae: 0.7453 - val_rmse: 1.2057
Epoch 6/50
504/504 [==============================] - 12s 24ms/step - loss: 1.4131 - mae: 0.7012 - rmse: 1.1572 - val_loss: 1.4966 - val_mae: 0.7353 - val_rmse: